Upravy 20/11/2019

In [18]:
import pandas as pd
import numpy as np
#from IPython.display import display

#pd.options.display.max_rows = 20000
#pd.options.display.max_columns = 100

Upravil som tabuľku vyberove_konania.csv a odstránil som z nej PSČ, ktoré sa v nej nachádzali, niektoré boli chybné a vnášalo to chaos. Podobne pokusy o to identifikovať kraj a podobne :)

In [19]:
PSC = pd.read_csv('./Data/PSC.csv', sep = ';')
ULICE = pd.read_csv('./Data/ULICE.csv', sep = ';')

statne_organizacie = pd.read_csv('./Data/statne_organizacie.csv', sep = ';')
vyberove_konania = pd.read_excel('vyberove_konania_20191119.xlsx', header=0, index_col=0)

### Urobíme adresy pre miesto výkonu a miesto úradu, nájdeme okres a kraj

Upravime tabuľku PSC.csv vo výberových konaniach, tak aby sme v nej vedeli vyhľadávať PSČ, ktoré sa vyskytujú vo výberových konaniach - odstránime medzery v PSČ. Zvládne to Bratislavu ale v Košiciach sú nejako divne okresy a kraje.

In [20]:
for index, row in PSC.iterrows():
    PSC.at[index, 'PSC'] = row['PSC'].replace(' ','')

PSC_upravene = PSC.copy()
for index, row in PSC_upravene.iterrows():
    PSC_upravene.at[index,'DOBEC'] = row['DOBEC'].replace(' ','')
    
for index, row in ULICE.iterrows():
    ULICE.at[index, 'PSC'] = str(row['PSC']).replace(' ','')

In [21]:
def najdi_podla_psc(psc):
    result = PSC[PSC['PSC'] == psc]
    if len(result.index) == 0:
        # Osetri Bratislavu
        if (psc[0] == '8'):
            dict = {
                '1' : 'I',
                '2' : 'II',
                '3' : 'III',
                '4' : 'IV',
                '5' : 'V'
            }
            return ['Bratislava','Bratislava '+dict[psc[1]],'BA']
        else:
            return ['','','']
    else:
        if result.index[0] > 1:
            result = result.iloc[0]
        return [result['DOBEC'], result['OKRES'], result['KRAJ']]
    
def najdi_psc_k_mestu(mesto):
    result = PSC[PSC['DOBEC'] == mesto]['PSC']
    if len(result.index) == 0:
        return ''
    else:
        if len(result.index[0] == 1):
            return str(result)
        else:
            return ''
        
def najdi_podla_mesta(mesto):
    result = PSC_upravene[PSC_upravene['DOBEC'] == mesto]
    if result.empty:
        return ['','']
    else:
        if result.index[0] > 1:
            result = result.iloc[0]
        return[result['OKRES'],result['KRAJ']]

In [22]:
empty_column = ['' for i in range(0,vyberove_konania.shape[0])]
vyberove_konania.insert(12,'sluzobny_urad_ulica',empty_column)
vyberove_konania.insert(13,'sluzobny_urad_mesto',empty_column)
vyberove_konania.insert(14,'sluzobny_urad_psc',  empty_column)
vyberove_konania.insert(15,'sluzobny_urad_okres',empty_column)
vyberove_konania.insert(16,'sluzobny_urad_kraj', empty_column)

Z bunky 'sluzobny_urad_sidlo' extrahujeme ulicu a mesto, podľa PSČ nájdeme okres a kraj. Košice ošetríme ručne v Exceli.

In [23]:
for index, row in vyberove_konania.iterrows():
    adresa = row['sluzobny_urad_sidlo'].split('\n')
    
    vyberove_konania.at[index,'sluzobny_urad_ulica'] = adresa[0]
    mesto, sep, cast = adresa[1].partition('-')
    vyberove_konania.at[index,'sluzobny_urad_mesto'] = mesto
    vyberove_konania.at[index,'sluzobny_urad_psc']   = adresa[2].replace(' ', '')
    
    mesto, okres, kraj = najdi_podla_psc(adresa[2])
    
    vyberove_konania.at[index,'sluzobny_urad_okres'] = okres
    vyberove_konania.at[index,'sluzobny_urad_kraj']  = kraj
    
vyberove_konania = vyberove_konania.drop(['sluzobny_urad_sidlo'], axis = 1)

In [24]:
vyberove_konania.insert(26,'miesto_vykonu_ulica', empty_column)
vyberove_konania.insert(27,'miesto_vykonu_mesto', empty_column)
vyberove_konania.insert(28,'miesto_vykonu_psc',   empty_column)
vyberove_konania.insert(29,'miesto_vykonu_okres', empty_column)
vyberove_konania.insert(30,'miesto_vykonu_kraj',  empty_column)

Nájdeme kraj a okres výkonu. Veci typu Bratislava-mestskacastNieco treba urobiť v Exceli.

In [25]:
# nahradime nan prazdnym stringom
vyberove_konania['miesto_vykonu'] = vyberove_konania['miesto_vykonu'].fillna('')

for index, row in vyberove_konania.iterrows():
    adresa = row['miesto_vykonu'].split('\n')
    ulica = ''
    mesto = ''
    psc   = ''
    
    if len(adresa) == 3:
        ulica = adresa[0].strip()
        mesto = adresa[1].strip()
        psc   = str(adresa[2]).strip().replace(' ', '')
    
    if len(adresa) == 2:
        ulica = adresa[0].strip()
        mesto = adresa[1].strip()
        psc   = ''
    
    if len(adresa) == 1:
        ulica = ''
        mesto = adresa[0].strip()
        psc   = ''

    vyberove_konania.at[index,'miesto_vykonu_ulica'] = ulica
    vyberove_konania.at[index,'miesto_vykonu_mesto'] = mesto
    vyberove_konania.at[index,'miesto_vykonu_psc'] = psc
    
    okres, kraj = najdi_podla_mesta(mesto)
    
    try:
        if okres == '':
            if psc != '':
                mesto, okres, kraj = najdi_podla_psc(psc)
    except:
        pass
        
    vyberove_konania.at[index,'miesto_vykonu_okres'] = okres
    vyberove_konania.at[index,'miesto_vykonu_kraj']  = kraj
    
vyberove_konania = vyberove_konania.drop(['miesto_vykonu'], axis = 1)

### Treba nájsť ešte IČO

Urobíme to inteligentne. Urobíme najprv dve pomocné štruktúry, v ktorých vieme rýchlo vyhľadávať, slovník štátnych organizácií, ku ktorým poznáme IČO a nie su v likvidácii, slová upravíme. Tabuľku všetkých organizácii z výberových konaní, ku ktorým chceme priradiť IČO, ak ho nájdeme raz, nemusíme ho už hľadať ďalších 1000-krát. 

Názvy nemusia sedieť presne, preto chceme nájsť najbližší string, k tomu čo hľadáme. Na vyhodnotenie podobnosti skúsime urobiť veľmi jednoduchú mieru usporiadaný 'skalárny súčin' zmysluplných slov. Ku každému názvu nájdeme najpodobnejší názov z inej tabuľky.

In [26]:
vyberove_konania.insert(10,'sluzobny_urad_ico',empty_column)

In [27]:
# Vytvori slovnik (Urad,ICO) pre vsetky zname statne organizacie, ktore nie su v likvidacii
ICOs = dict()

for index, row in statne_organizacie.iterrows():
    if not row['NAZOV'] in ICOs and not 'v likvidácii' in row['NAZOV']:
        ICOs[row['NAZOV'].casefold().strip()] = row['ICO']

In [28]:
treba_najst = dict()

for index, row in vyberove_konania.iterrows():
    if not row['sluzobny_urad'] in treba_najst:
        if row['sluzobny_urad'] in ICOs:
            treba_najst[row['sluzobny_urad']] = ICOs[row['sluzobny_urad']]
        else:
            treba_najst[row['sluzobny_urad']] = None
        
# Ulozime si tie urady, pre ktore sme nenasli ostru zhodu
nenajdene = [urad for urad in treba_najst if treba_najst[urad] == None]

In [29]:
# Rozmeni nazov urad na slova a zahodi interpunkcne znamienka atd ...
def parse(word):
    slovak_alphabet = 'aáäbcčdďeéfghiíjklĺľmnňoóôpqrŕsštťuúvwxyýzž1234567890'
    result = ''
    for char in word:
        if char in slovak_alphabet:
            result = result + char
    return result
    
def parse_and_break(sentence):
    sentence = sentence.casefold().strip()
    sentence = sentence.split(' ')
    return [parse(word) for word in sentence if parse(word) != '']

# Zadefinuje skalarny sucin nazvov
def skalarny_sucin(string_a,string_b):
    slova_a = parse_and_break(string_a)
    slova_b = parse_and_break(string_b)

    if len(slova_a)>len(slova_b):
        slova_c = slova_a.copy()
        slova_a = slova_b.copy()
        slova_b = slova_c.copy()
    
    result = 0
    for index, slovo in enumerate(slova_a):
        if slovo == slova_b[index]:
            result += 1
        else:
            break
            
    return result

def najdi_ICO(hladany_urad):
    urady = [[urad,ICOs[urad],0] for urad in ICOs]

    for index, urad in enumerate(urady):
        urady[index][2] = skalarny_sucin(hladany_urad, urad[0])

    urady.sort(key=lambda tup: tup[2], reverse = True)
    
    # Skontroluje ci sme nasli ostro najlepsi vysledok alebo je viac kandidatov
    # ak je viac kandidatiov napr. nieco typu v nazve je Okresny sud a rozlisit sa to da iba podla dalsieho stlpca
    # tak radsej nevrati nic
    
    if (urady[0][2]>urady[1][2]):
        return urady[0][1]
    else:
        return None

In [30]:
co_sme_nasli = dict()
for urad in nenajdene:
    co_sme_nasli[urad] = najdi_ICO(urad)
    treba_najst[urad]  = najdi_ICO(urad)

In [31]:
este_stale_nenajdene = [urad for urad in co_sme_nasli if co_sme_nasli[urad] == None]
print('Kolko treba dohladat manualne? ',len(este_stale_nenajdene))
# Urady ktore treba dohladat manualne ulozime do subory
# Napriklad preto ze maju ine nazvy typicky v jednej tabulke
# Regionalny urad verejneho zdravotnictva Lucenec a Regionalny urad vereneho zdravotnictva so sidlom v Lucenci
for urad in este_stale_nenajdene:
    print('co_sme_nasli[\''+urad+'\']=')

Kolko treba dohladat manualne?  35
co_sme_nasli['Regionálny úrad verejného zdravotníctva Čadca']=
co_sme_nasli['Regionálny úrad verejného zdravotníctva Liptovský Mikuláš']=
co_sme_nasli['Regionálny úrad verejného zdravotníctva Martin']=
co_sme_nasli['Regionálny úrad verejného zdravotníctva Lučenec']=
co_sme_nasli['Regionálny úrad verejného zdravotníctva Trenčín']=
co_sme_nasli['Inšpektorát práce Trenčín']=
co_sme_nasli['Regionálny úrad verejného zdravotníctva Zvolen']=
co_sme_nasli['Regionálny úrad verejného zdravotníctva Michalovce']=
co_sme_nasli['Ústredný inšpektorát Slovenskej obchodnej inšpekcie']=
co_sme_nasli['Regionálny úrad verejného zdravotníctva Levice']=
co_sme_nasli['Úrad verejného zdravotníctva SR Bratislava']=
co_sme_nasli['Regionálny úrad verejného zdravotníctva Galanta']=
co_sme_nasli['Regionálny úrad verejného zdravotníctva, Humenné']=
co_sme_nasli['Regionálny úrad verejného zdravotníctva Komárno']=
co_sme_nasli['Regionálny úrad verejného zdravotníctva Vranov nad Topľ

In [32]:
co_sme_nasli['Úrad pre reguláciu hazardných hier']='52265021'
co_sme_nasli['Inšpektorát práce Žilina']='35993499'
co_sme_nasli['Inšpektorát práce Trenčín']='35627620'
co_sme_nasli['Úrad verejného zdravotníctva SR Bratislava']='00607223'
co_sme_nasli['Ústredný inšpektorát Slovenskej obchodnej inšpekcie']='17331927'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Žiar nad Hronom']='17336104'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Čadca']='17335493'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Liptovský Mikuláš']='00607215'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Martin']='17335621'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Komárno']='17335655'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Zvolen']='17335710'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Michalovce']='17335680'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Nitra']='17336031'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Nové Zámky']='17336121'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Banská Bystrica']='00606979'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Bardejov']='00611026'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Spišská Nová Ves']='00610984'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Stará Ľubovňa']='17335451'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Trnava']='00610933'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Vranov nad Topľou']='17335787'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Trebišov']='17335434'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Dunajská Streda']='17335981'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Galanta']='00610917'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Humenné']='00611018'
co_sme_nasli['Regionálny úrad verejného zdravotníctva pre územný obvod okresov Senica a Skalica']='00611034'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Prešov']='00610992'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Žilina']='17335876'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Rožňava']='17335957'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Levice']='00610909'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Košice']='00606723'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Topoľčany']='17335761'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Trenčín']='00610968'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Lučenec']='17335558'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Svidník']='17335914'
co_sme_nasli['Regionálny úrad verejného zdravotníctva Rimavská Sobota']='00611000'

Nič nové hľadať netreba.

In [33]:
for urad in co_sme_nasli:
    treba_najst[urad] = co_sme_nasli[urad]
    
for index, row in vyberove_konania.iterrows():
    vyberove_konania.at[index,'sluzobny_urad_ico'] = treba_najst[row['sluzobny_urad']]

In [34]:
vyberove_konania.to_excel('vyberove_konania_20191119_upravene .xlsx')

In [36]:
vyberove_konania.to_pickle('vyberove_konania_20191119_upravene.pkl')

In [29]:
vyberove_konania.to_csv('vyberove_konania_20191119_upravene.csv', sep=';')